In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
class node(object):
    
    def __init__(self,state,inbound=None,outbound=None):
        self.state=state
        self.inbound={}
        self.outbound={}
        
    def __str__(self):
        string="State: "+str(self.state)
        return string
    
    def addInbound(self,inboundNode,inboundArc):
        self.inbound[inboundNode]=inboundArc
        
    def addOutbound(self,outboundNode,outboundArc):
        self.outbound[outboundNode]=outboundArc


In [3]:
class layer(object):
    
    def __init__(self, layer_no):
        self.layer_no=layer_no
        self.nodes={}
        
    def __str__(self):
        string="Layer no. {}, containing {} nodes".format(self.layer_no,len(self.nodes))
        return string
    
    def addNode(self,node):
        self.nodes[node.state]=node
        
    def getNodes(self):
        return self.nodes

In [4]:
class arc(object):
    
    def __init__(self, head, tail, arctype, length):
        self.head=head
        self.tail=tail
        self.arctype=arctype
        self.length=length
    
    def __str__(self):
        string="Arc type: {}, Length: {}, Headnode: {}, Tailnode: {}".format(self.arctype,self.length,self.head.state, self.tail.state)
        return string

In [5]:
n=6
U=10

np.random.seed(78440)
    
cvals=np.random.randint(1,U,n)
wvals=np.random.randint(1,U,n)
W=sum(wvals)/2
d=[0,1]

In [6]:
n1=25
U1=50
    
cvals1=np.random.randint(1,U1,n1)
wvals1=np.random.randint(1,U1,n1)
W1=sum(wvals1)/2
d1=[0,1]

def read_instance(address = 'instance.csv'):
    data = pd.read_csv(address)
    c = data['c'].values
    w = data['w'].values
    W = data['W'].values[0]
    return c,w,W

In [7]:
def Algorithm_1(n,cvals,wvals,W,d):
    
    root_node=node(0)
    L1=layer(1)
    L1.addNode(root_node)
    
    arcList=[]
    all_layers={}
    all_layers[0]=L1
    
    #Iterate through each layer
    for x in range(1,n):
        #Create next layer
        currLayer=all_layers[x-1]
        all_layers[x]=layer(x+1)
        nextLayer=all_layers[x]
        #Iterate through each node and action decision
        for nodeInstance in currLayer.nodes.values():
            for decision in d:
                newstate=nodeInstance.state+(wvals[x-1]*decision)
                #Feasability check
                #Check if the new state already exists in the layer
                if newstate<=W:
                    if newstate in nextLayer.nodes:
                        #Add arc into arclist
                        new_arc=arc(nodeInstance,nextLayer.nodes[newstate],decision,cvals[x-1]*decision)
                        arcList.append(new_arc)
                        #Add arc into node inbound and outbound
                        nodeInstance.addOutbound(new_arc)
                        nextLayer.nodes[newstate].addInbound(new_arc)
                    else:
                        #Create new node, add arc to arclist
                        new_node=node(newstate)
                        nextLayer.nodes[newstate]=new_node
                        new_arc=arc(nodeInstance,new_node,decision,cvals[x-1]*decision)
                        arcList.append(new_arc)
                        #Add arc to node inbound and outbound
                        nodeInstance.addOutbound(new_arc)
                        new_node.addInbound(new_arc) 
        
    #Changed from W because Rachel said so
    terminal_node=node(-999)
    Lt=layer(n+1)
    Lt.nodes[-999]=terminal_node
    all_layers[n]=Lt
    
    for nodeInstance in all_layers[n-1].nodes.values():
        for decision in d:
            newstate=nodeInstance.state+(wvals[n-1]*decision)
            if newstate<=W:
                new_arc=arc(nodeInstance,terminal_node,decision,cvals[n-1]*decision)
                arcList.append(new_arc)
                nodeInstance.addOutbound(new_arc)
                terminal_node.addInbound(new_arc)
                
    terminal_node.addOutbound(arc(terminal_node,node(0),0,0))
                
    return all_layers, arcList

In [8]:
def Algorithm_2(all_layers,arcList,n):
    #Final reduction algorithm here we go
    reduced_layers=all_layers.copy()
    reduced_arcList=arcList.copy()
    #Starting from the second last layer moving up
    for x in range(n-1,0,-1):
        for node_1 in reduced_layers[x].nodes.values():
            to_remove=[]
            for node_2 in reduced_layers[x].nodes.values():
                if(node_1 is not node_2):
                    
                    #If only one outgoing arc from each node pair
                    if (len(node_1.outbound)==1 and len(node_2.outbound)==1):
                        if(node_1.outbound[0].arctype is node_2.outbound[0].arctype and node_1.outbound[0].tail is node_2.outbound[0].tail):
                            if (node_2.outbound[0] in reduced_arcList):
                                reduced_arcList.remove(node_2.outbound[0])
                                node_2.outbound.remove(node_2.outbound[0])
                            for incomingArc in node_2.inbound:
                                incomingArc.tail=node_1                             
                            to_remove.append(node_2)
                    
                    #If two outgoing arcs from each node pair       
                    elif (len(node_1.outbound)==2 and len(node_2.outbound)==2):
                        if(node_1.outbound[0].arctype is node_2.outbound[0].arctype and node_1.outbound[0].tail is node_2.outbound[0].tail and node_1.outbound[1].arctype is node_2.outbound[1].arctype and node_1.outbound[1].tail is node_2.outbound[1].tail):
                            if (node_2.outbound[0] in reduced_arcList):
                                reduced_arcList.remove(node_2.outbound[0])
                                node_2.outbound.remove(node_2.outbound[0])
                            #Once first arc removed, only one left so re-index as 0
                            if (node_2.outbound[0] in reduced_arcList):
                                reduced_arcList.remove(node_2.outbound[0])
                                node_2.outbound.remove(node_2.outbound[0])
                            for incomingArc in node_2.inbound:
                                incomingArc.tail=node_1                             
                            to_remove.append(node_2)     
                            
            #Remove nodes before moving onto next starting node pair seed                                         
            for remove_node in to_remove:
                if (remove_node in reduced_layers[x].nodes):
                    reduced_layers[x].nodes.remove(remove_node)
                    
    return reduced_layers, reduced_arcList

In [9]:
[cvals2,wvals2,W2]=read_instance()
n2=len(cvals2)

In [10]:
start1=time.time()
[all_layers,arcList]=Algorithm_1(n2,cvals2,wvals2,W2,d)
end1=time.time()
#[reduced_layers,reduced_arcList]=Algorithm_2(all_layers.copy(),arcList.copy(),n2)
end2=time.time()

In [11]:
#Just printing stuff, no biggie
    
print("Algo 1 Runtime: {}ms".format(1000*(end1-start1)))
print("Algo 2 Runtime: {}ms".format(1000*(end2-end1)))
print("Total Runtime: {}ms".format(1000*(end2-start1)))

for layer in all_layers.values():
    print("Layer no: {}".format(layer.layer_no))
    print("Nodes in layer: {}".format(len(layer.nodes.values())))
    '''for node in layer.nodes.values():
        print("Node: {}".format(node))
        for outArc in node.outbound:
            print(outArc)
    print("")'''

Algo 1 Runtime: 357.9597473144531ms
Algo 2 Runtime: 0.0ms
Total Runtime: 357.9597473144531ms
Layer no: 1
Nodes in layer: 1
Layer no: 2
Nodes in layer: 2
Layer no: 3
Nodes in layer: 4
Layer no: 4
Nodes in layer: 6
Layer no: 5
Nodes in layer: 12
Layer no: 6
Nodes in layer: 21
Layer no: 7
Nodes in layer: 38
Layer no: 8
Nodes in layer: 57
Layer no: 9
Nodes in layer: 76
Layer no: 10
Nodes in layer: 93
Layer no: 11
Nodes in layer: 114
Layer no: 12
Nodes in layer: 133
Layer no: 13
Nodes in layer: 150
Layer no: 14
Nodes in layer: 170
Layer no: 15
Nodes in layer: 178
Layer no: 16
Nodes in layer: 199
Layer no: 17
Nodes in layer: 205
Layer no: 18
Nodes in layer: 219
Layer no: 19
Nodes in layer: 230
Layer no: 20
Nodes in layer: 243
Layer no: 21
Nodes in layer: 265
Layer no: 22
Nodes in layer: 282
Layer no: 23
Nodes in layer: 295
Layer no: 24
Nodes in layer: 314
Layer no: 25
Nodes in layer: 323
Layer no: 26
Nodes in layer: 325
Layer no: 27
Nodes in layer: 337
Layer no: 28
Nodes in layer: 352
Layer 